In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import linear, relu, sigmoid
from scipy.ndimage import shift
%matplotlib widget

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [2]:
df = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
y = df['label'].to_numpy()
X = df.iloc[:,1:].to_numpy()
print(f'x-shape: {X.shape}, y-shape: {y.shape}')

x-shape: (42000, 784), y-shape: (42000,)


In [3]:
# function to augment (shift image)
def shift_image(image, dx, dy):
    image = image.reshape(28,28)
    shifted_image = shift(image, [dy, dx], mode='constant')
    return shifted_image.reshape(-1)

# create augmented dataset
x_train_augmented = [image for image in X]
y_train_augmented = [image for image in y]

# shift right, left, up, then down
for dx, dy in ((1,0), (-1,0), (0,1), (0,-1)):
    for image, label in zip(X, y):
        x_train_augmented.append(shift_image(image, dx, dy))
        y_train_augmented.append(label)
# shuffle the dataset
shuffle_idx = np.random.permutation(len(x_train_augmented))
x_train_augmented = np.array(x_train_augmented)[shuffle_idx]
y_train_augmented = np.array(y_train_augmented)[shuffle_idx]

In [4]:
model = Sequential(
    [
        tf.keras.Input(shape=(784,)),
        Dense(units=32, activation='relu', name='L1'),
        Dense(units=15, activation='relu', name='L2'),
        Dense(units=15, activation='relu', name='L3'),
        Dense(units=25, activation='relu', name='L4'),
        Dense(units=10, activation='linear', name='L5')
        
    ]
)

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 L1 (Dense)                  (None, 32)                25120     
                                                                 
 L2 (Dense)                  (None, 15)                495       
                                                                 
 L3 (Dense)                  (None, 15)                240       
                                                                 
 L4 (Dense)                  (None, 25)                400       
                                                                 
 L5 (Dense)                  (None, 10)                260       
                                                                 
Total params: 26,515
Trainable params: 26,515
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(learning_rate=.001,),
    metrics='accuracy'
)

model.fit(
    X,y,
    epochs=200
)

Epoch 1/200
1313/1313 [==============================] - 10s 3ms/step - loss: 1.5179 - accuracy: 0.5747
Epoch 2/200
1313/1313 [==============================] - 4s 3ms/step - loss: 0.4669 - accuracy: 0.8831
Epoch 3/200
1313/1313 [==============================] - 4s 3ms/step - loss: 0.3272 - accuracy: 0.9158
Epoch 4/200
1313/1313 [==============================] - 4s 3ms/step - loss: 0.2641 - accuracy: 0.9288
Epoch 5/200
1313/1313 [==============================] - 4s 3ms/step - loss: 0.2250 - accuracy: 0.9391
Epoch 6/200
1313/1313 [==============================] - 4s 3ms/step - loss: 0.2044 - accuracy: 0.9440
Epoch 7/200
1313/1313 [==============================] - 5s 4ms/step - loss: 0.1798 - accuracy: 0.9509
Epoch 8/200
1313/1313 [==============================] - 4s 3ms/step - loss: 0.1670 - accuracy: 0.9540
Epoch 9/200
1313/1313 [==============================] - 4s 3ms/step - loss: 0.1546 - accuracy: 0.9562
Epoch 10/200
1313/1313 [==============================] - 4s 3ms/step - 

# Prediction:

In [7]:
# Evaluate the model performance
test_loss, test_acc = model.evaluate(x=X, y=y)
# Print out the model accuracy 
print('\nTest accuracy:', test_acc)

1313/1313 [==============================] - 3s 2ms/step - loss: 0.0494 - accuracy: 0.9893

Test accuracy: 0.9892857074737549


# Test data output

In [8]:
df_test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
X_test = df_test.to_numpy()

prediction = model.predict(X_test)

final_predict = pd.DataFrame({'ImageID':np.arange(1, (X_test.shape[0] + 1)),
             'Label':np.argmax(prediction, axis=1)})

final_predict.to_csv('/kaggle/working/submission.csv',index=False)

875/875 [==============================] - 2s 2ms/step
